In [1]:
# 进行机器人基本聊天的编写

In [26]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
# download thw necessary English package
nltk.download('punkt')
# things we need for tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

# import our chat-bot intents file
import json

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\housh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
stemer = LancasterStemmer()

# 加载我们的json数据
with open('D:\Project_coding\Tensorflow\Chat\intents.json') as json_data:
    intents = json.load(json_data)

In [31]:
# create our training data
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in sentence
        w = nltk.word_tokenize(pattern)
        # add to out words list
        words.extend(w)
        # add to documents in out corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [33]:
# stem and lower each word and remove duplicates
words = [stemer.stem(w.lower())
         for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print(len(documents), "documents", documents)
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)

27 documents [(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['What', 'hours', 'are', 'you', 'open', '?'], 'hours'), (['What', 'are', 'your', 'hours', '?'], 'hours'), (['When', 'are', 'you', 'open', '?'], 'hours'), (['Which', 'mopeds', 'do', 'you', 'have', '?'], 'mopeds'), (['What', 'kinds', 'of', 'mopeds', 'are', 'there', '?'], 'mopeds'), (['What', 'do', 'you', 'rent', '?'], 'mopeds'), (['Do', 'you', 'take', 'credit', 'cards', '?'], 'payments'), (['Do', 'you', 'accept', 'Mastercard', '?'], 'payments'), (['Are', 'you', 'cash', 'only', '?'], 'payments'), (['Are', 'you', 'open', 'today', '?'], 'opentoday'), (['When', 'do', 'you', 'open', 'today', '?'], 'opentoday'), (['What', 'are', 'your', 'hours',

In [40]:
# create our training data
training = []
output = []
# create an empty array for out output
output_empty = [0] * len(classes)

# training set ,bag of words for each sentence
for doc in documents:
    # initalize out bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# print(len(training), "training", training)

# create train and test lists
train_x = list(training[:, 0])
train_y = list(training[:, 1])

# print(len(train_x), "train_x", train_x)
# print(len(train_y), "train_y", train_y)

In [42]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training （apply gradient descent algorithm）
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

---------------------------------
Run id: 54K0EZ
Log directory: tflearn_logs/


INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.


---------------------------------
Training samples: 27
Validation samples: 0
--


Training Step: 1  | time: 0.694s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 -- iter: 08/27


Training Step: 2  | total loss: 1.97751 | time: 0.701s
| Adam | epoch: 001 | loss: 1.97751 - acc: 0.0000 -- iter: 16/27


...

Training Step: 3997  | total loss: 0.60256 | time: 0.007s
| Adam | epoch: 1000 | loss: 0.60256 - acc: 0.9301 -- iter: 08/27


Training Step: 3998  | total loss: 0.55939 | time: 0.015s
| Adam | epoch: 1000 | loss: 0.55939 - acc: 0.9371 -- iter: 16/27


Training Step: 3999  | total loss: 0.51471 | time: 0.022s
| Adam | epoch: 1000 | loss: 0.51471 - acc: 0.9434 -- iter: 24/27


Training Step: 4000  | total loss: 0.47455 | time: 0.028s
| Adam | epoch: 1000 | loss: 0.47455 - acc: 0.9490 -- iter: 27/27
--


INFO:tensorflow:D:\Project_coding\Tensorflow\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [43]:
def clean_up_sentense(sentence):
    # tokenize the pattern
    sentense_words = nltk.word_tokenize(sentence)
    # stem each word
    sentense_words = [stemer.stem(word.lower()) for word in sentense_words]
    return sentense_words


# return bag of words array :0 or 1 for each word in the bg that exists in the sentense
def bow(sentence, word, show_details=False):
    # tokenize the pattern
    sentence_words = [stemer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bg that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentense(sentence)
    # bag of words
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag : %s" % w)
    return np.array(bag)

In [44]:
p =bow("is your shop open today?" , words)
print(p)
print(classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 1 0]
['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']


In [45]:
print(model.predict([p]))

[[  1.92591498e-04   6.42503583e-05   2.63959523e-02   1.61695220e-02
    8.22831810e-01   2.06097233e-04   2.08797446e-03   2.83839967e-04
    1.31768003e-01]]


In [46]:
# save all of our data structures
import pickle

pickle.dump({'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y}, open("training_data", "wb"))
